# Unsupervised Learning

The first part of this exercise will be dedicated to implementing the *K*-means **clustering** algorithm and applying it to compress an image. In the second part, the principal component analysis (PCA) will be used in order to find a low-dimensional representation of face images. 

## 1 *K*-means clustering algorithm

Powerfull unsupervised learning algorithm here will be used for image compression. 

### 1.1 Implementing *K*-means
